In [1]:
!pip install aixd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.5/169.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 861.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 54.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cusparselt-cu12
    Found existing installation: nvidia-cusparselt-cu12 0.7.1
    Uninstalling nvidia-cusparselt-cu12-0.7.1:
      Successfully uninstalled nvidia-cus

In [2]:
# Install torch, torchvision, torchaudio with CUDA 12.6 support
!pip install torch==2.8.0+cu126 torchvision==0.23.0+cu126 torchaudio==2.8.0+cu126 --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.8/821.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 MB 6.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cusparselt-cu12
    Found existing installation: nvidia-cusparselt-cu12 0.6.3
    Uninstalling nvidia-cusparselt-cu12-0.6.3:
      Successfully uninstalled nvidia-cusparselt-cu12-0.6.3
  Attempting uninstall: triton
    Found existing installation: triton 3.3.1
    Uninstalling triton-3.3.1:
      Successfully uninstalled triton-3.3.1
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.26.2
    Uninstalling nvidia-nccl-cu12-2.26.2:
      Successfully uninstalled nv

### 2. Synthetic Data Generation

now we hand over the LHS sampled input feature value list to Rhino / GH to obtain performance values per design instance.

now first do some inspection of the performance attribute values


In [5]:
import numpy as np
import pandas as pd
from scipy.stats import qmc

df_lhs = pd.read_csv("lhs_samples_results.csv")

df_lhs.describe()


,Nfrequ1,Nfrequ2,Nfrequ3,max_disp_cm,Mass_Kg,util_average
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.863000,2.983400,4.302100,129.343545,193264.102000,2.242958
std,1.030909,1.554961,1.919497,673.520009,63868.629612,5.739388
min,0.100000,0.300000,0.400000,0.570000,76100.000000,0.207163
25%,1.100000,1.775000,2.800000,2.565000,140660.000000,0.488279
50%,1.600000,2.900000,4.300000,5.130000,191286.000000,0.763154
75%,2.500000,3.700000,5.700000,16.530000,235166.750000,1.412177
max,7.800000,9.300000,11.400000,12111.075000,401078.000000,86.297872


### 3. Data set creation

#### Define your design parameter and perfromance attributes


In [6]:
from aixd.data.data_blocks import DesignParameters, PerformanceAttributes
from aixd.data.data_objects import DataReal, DataInt, DataCategorical
from aixd.data.domain import Interval, Options

# Defining Design Parameter
obj_list = [
    DataReal(name="A", dim=1, domain = Interval(0.0, 5.0)),
    DataReal(name="mu", dim=1, domain = Interval(0.0, 1.0)),
    DataReal(name="sigma", dim=1, domain = Interval(0.05, 1.0)),
    DataInt(name="num_piers", dim=1, domain = Interval(1, 6)),
    DataReal(name="pier_h", dim=1, domain = Interval(-6.0, 0.0), unit="m"),
    DataReal(name="shell_cross_section", dim=1, domain = Interval(10.0, 30.0), unit="cm"),
    DataReal(name="pier_diameter", dim=1, domain = Interval(20.0, 60.0), unit="cm"),
    DataCategorical(name="Concrete", dim=1, domain = Options(["C30/37", "C40/50", "C60/75"])),
    ]

design_parameters = DesignParameters(name="design_parameters", dobj_list=obj_list)


/usr/local/lib/python3.12/dist-packages/aixd/data/domain.py:575: UserWarning: Expected to get an object of class <Interval> with type integer, but got type real instead. Creating a new domain object with type to integer.
  warnings.warn(


And similar we define the performance attributes:

In [8]:

obj_list = [
    DataReal(name="Nfrequ1", dim=1, unit="Hz"),
    DataReal(name="Nfrequ2", dim=1, unit="Hz"),
    DataReal(name="Nfrequ3", dim=1, unit="Hz"),
    DataReal(name="max_disp_cm", dim=1, unit="cm"),
    DataReal(name="Mass_Kg", dim=1, unit="kg"),
    DataReal(name="util_average", dim=1, unit="-"),
]

performance_attributes = PerformanceAttributes(name="performance_attributes", dobj_list=obj_list)


/usr/local/lib/python3.12/dist-packages/aixd/data/data_objects.py:433: UserWarning: Expected to get an object of class <Interval> as a domain. One created automatically with range 0 to 100
  warnings.warn("Expected to get an object of class <Interval> as a domain. One created automatically with range 0 to 100")


#### Create your dataset object

Using the defined design parameters and performance attributes, we initialize the dataset object. Here we just define the structure of the data set, so far no actual data is saved in the data set. The function also creates a dataset folder in the directory of the notebook.


In [9]:
from aixd.data.dataset import Dataset

dataset = Dataset(root_path=None, name="Bridge_Design", design_par=design_parameters, perf_attributes=performance_attributes, overwrite=True)

[INFO - AIXD.dataset] Dataset object is saved at /content/Bridge_Design/dataset_object.json


#### Import data to your dataset object

If we have data available we can directly import our data into the dataset from our dataframe. Here we save our data from the dataFrame to the dataset object we have created before.


In [12]:
# Load your two CSVs
df_samples = pd.read_csv("lhs_samples.csv")
df_results = pd.read_csv("lhs_samples_results.csv")

# Combine the two csv
df = pd.concat([df_samples.reset_index(drop=True), df_results.reset_index(drop=True)], axis=1)

# Save the combined dataset
df.to_csv("bridge_data_set.csv", index=False)

# Create the XAID dataset object
dataset.import_data_from_df(df)

[WARNING - AIXD.callbacks] Error column not found. Adding an all zero to the end
[INFO - AIXD.dataset] 1000 design parameters imported and stored in design_parameters/dp_0000.pkl
[INFO - AIXD.dataset] 1000 performance attributes imported and stored in performance_attributes/pa_0000.pkl
[INFO - AIXD.dataset] Dataset object is saved at /content/Bridge_Design/dataset_object.json
[INFO - AIXD.dataset] Data import finished. Updating range of performance attributes
[INFO - AIXD.dataset] Loaded a total of 1000 samples from 1 files in 0.0048792362213134766 seconds
[INFO - AIXD.data-objects] * Updating domain for Nfrequ1: The domain spans the interval [0.0, 1.0], but the data spans the interval [0.1, 7.8] (floored/ceiled to 3 decimal). Updating the domain to match the data. The new domain is Interval(0.1, 7.8).
[INFO - AIXD.data-objects] * Updating domain for Nfrequ2: The domain spans the interval [0.0, 1.0], but the data spans the interval [0.3, 9.3] (floored/ceiled to 3 decimal). Updating the

In [13]:
dataset.data["design_parameters"].head()

,uid,A,mu,sigma,num_piers,pier_h,shell_cross_section,pier_diameter,Concrete
0,0,0.57113,0.195561,0.432034,1,-0.79,24,31,C30/37
1,1,4.56107,0.254872,0.803872,1,-4.97,23,45,C30/37
2,2,3.757783,0.747773,0.832273,6,-1.52,18,29,C30/37
3,3,1.123227,0.834029,0.720802,6,-2.75,22,40,C40/50
4,4,1.529229,0.096317,0.928042,1,-5.5,12,46,C40/50


In [14]:
dataset.data["performance_attributes"].head()

,uid,Nfrequ1,Nfrequ2,Nfrequ3,max_disp_cm,Mass_Kg,util_average,error
0,0,1.0,1.3,2.0,71.82,184320,3.764846,0
1,1,1.0,1.4,3.5,11.4,312402,1.374871,0
2,2,2.3,3.6,5.7,2.565,220215,0.613419,0
3,3,2.5,4.3,6.2,0.57,184431,0.354233,0
4,4,1.0,1.2,2.5,35.91,105400,1.78484,0


In [15]:
dataset.update_obj_domains()

[INFO - AIXD.data-objects] * Updating domain for A: The domain spans the interval [0.0, 5.0], but the data spans the interval [0.003, 4.997] (floored/ceiled to 3 decimal). Updating the domain to match the data. The new domain is Interval(0.003, 4.997).
[INFO - AIXD.data-objects] * Updating domain for mu: OK (no update)
[INFO - AIXD.data-objects] * Updating domain for sigma: OK (no update)
[INFO - AIXD.data-objects] * Updating domain for num_piers: The domain spans the interval [1, 6], but the data spans the interval [0.0, 6.0] (floored/ceiled to 3 decimal). Updating the domain to match the data. The new domain is Interval(0, 6).
[INFO - AIXD.data-objects] * Updating domain for pier_h: The domain spans the interval [-6.0, 0.0], but the data spans the interval [-6.0, -0.01] (floored/ceiled to 3 decimal). Updating the domain to match the data. The new domain is Interval(-6.0, -0.01).
[INFO - AIXD.data-objects] * Updating domain for shell_cross_section: OK (no update)
[INFO - AIXD.data-obj

### 3. Data exploration

Next we can use the data visualization functionalities of the plotter to understand our data. For that we need to set up our plotter object

In [16]:
from aixd.visualisation.plotter import Plotter

plotter = Plotter(dataset)

[INFO - AIXD.plotter] 
Plotter: Information
--------------------
The following block names and variable names are available as arguments for plotting methods:
    Block "design_parameters" (Design Parameters):
    -> Variables: "A", "mu", "sigma", "num_piers", "pier_h", "shell_cross_section", "pier_diameter", "Concrete"
    Block "performance_attributes" (Performance Attributes):
    -> Variables: "Nfrequ1", "Nfrequ2", "Nfrequ3", "max_disp_cm", "Mass_Kg", "util_average"
Data from dataset blocks will be plotted in the original data domain.
Data from datamodule blocks will be plotted in the transformed domain (if `transformed=True`) or in the original domain, but after transformations (if `transformed=False`).



Helpful is for example to visualize the individual aswell as the pair-wise variable distributions.


In [17]:
plotter.distrib_attributes(block="design_parameters", per_column=True, sub_figs=True)

In [18]:
plotter.distrib_attributes(block="performance_attributes", per_column=True, sub_figs=True)

In [19]:
plotter.contours2d(blocks=["design_parameters"])

In [20]:
plotter.contours2d(blocks=["performance_attributes"])

In [21]:
plotter.distrib_attributes2d(blocks=["design_parameters", "performance_attributes"])

[WARNING - AIXD.plotter] Too many sub-figures to plot. Clipping to 49 sub-figures. To suppress this warning, set Plotter.MAX_SUBFIGURES to a larger value.


In [22]:
plotter.correlation(blocks=["design_parameters", "performance_attributes"])

/usr/local/lib/python3.12/dist-packages/aixd/visualisation/plotter.py:739: UserWarning:

Columns ['Concrete'] are not numeric and will be ignored.



### 4. Model set-up, training and evaluation

The next step is to set-up the model. In this example we train a conditional variational autoencoder to be able to function both as a surrogate model (for an input X predicting Y) aswell as an performance-oriented generative model (for defined constraints for Y generate valid Xs).

#### Set-up

First we define the input and output of our model. In this case the inputs are our design parameters (X) and our outputs the performance attributes.


In [23]:
inputML = dataset.design_par.names_list
outputML = dataset.perf_attributes.names_list

Next, we need to create a datamodule from our dataset.

In [24]:
from aixd.mlmodel.data.data_loader import DataModule

datamodule = DataModule.from_dataset(dataset, input_ml_names=inputML, output_ml_names=outputML, batch_size=1000)

[INFO - AIXD.data-objects] * Updating domain for Concrete: OK (no update)


Now, we can define the structure of our model. We first import the model we want to train and then initialize this model from our datamodule. We here define the number of layers (here:3) and the individual widths of each layer (here: 16,8,4). Furthermore the number of latent dimensions has to be chosen (here:1).

In [33]:
from aixd.mlmodel.architecture.cond_ae_model import CondAEModel

loss_weights = {"x": 1.0, "y": 1, "kl": 0.1}
cae = CondAEModel.from_datamodule(datamodule, layer_widths=[128, 128, 64, 64, 32, 16], latent_dim=3, loss_weights=loss_weights)

#### Training

Now that the model structure is set-up we train the model. Meaning we fit the model to our training data.Here we select to train maximum for 100 epochs and turn on early stopping.




In [34]:
import warnings

max_epochs = 20000

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    cae.fit(datamodule, name_run="Run1", max_epochs=max_epochs, flag_early_stop=True)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | encoder | Encoder | 76.6 K | train
1 | decoder | Decoder | 77.0 K | train
--------------------------------------------
153 K     Trainable params
0         Non-trainable params
153 K     Total params
0.614     Total estimated model params size (MB)
213       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

#### Evaluation

The third step of this chapter is the evaluation of the before trained model. We first import the model and then visually evaluate it. The model evaluation is important, in order to discern how successful the training has been, and if the architecture may benefit from changes.

To load the model we just need to provide the path to the checkpoint.

In [35]:
path = f"{CondAEModel.CHECKPOINT_DIR}/last.ckpt"
cae = CondAEModel.load_model_from_checkpoint(path)

To visually evaluate the trained model we have multiple helpful functionalities in the plotter. First we need to initialize the plotter again. This time additional to the dataset we also need to provide the model that we want to evaluate to the plotter object.

In [39]:
from aixd.visualisation.plotter import Plotter

plotter = Plotter(dataset=dataset, model=cae)

[INFO - AIXD.plotter] 
Plotter: Information
--------------------
The following block names and variable names are available as arguments for plotting methods:
    Block "design_parameters" (Design Parameters):
    -> Variables: "A", "mu", "sigma", "num_piers", "pier_h", "shell_cross_section", "pier_diameter", "Concrete"
    Block "performance_attributes" (Performance Attributes):
    -> Variables: "Nfrequ1", "Nfrequ2", "Nfrequ3", "max_disp_cm", "Mass_Kg", "util_average"
Data from dataset blocks will be plotted in the original data domain.
Data from datamodule blocks will be plotted in the transformed domain (if `transformed=True`) or in the original domain, but after transformations (if `transformed=False`).



The below figure shows the predicted values plotter against the true values from the validation dataset. A perfect fit would mean that all scatter points lie on the 45° line. By normalizing the data we archive the error in the original units.


In [40]:
plotter.attributes_obs_vs_pred(block="inputML", datamodule=datamodule, transformed=False, n_cols=2)

[INFO - AIXD.plotter] Overwriting datamodule with the provided one: {Train dataloader: size=800}
{Validation dataloader: size=100}
{Test dataloader: size=100}
{Predict dataloader: None}


/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 800 for training dataloader.

/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 100 for validation dataloader.

/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 100 for testing dataloader.

/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 100 for validation dataloader.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [41]:
plotter.attributes_obs_vs_pred(block="outputML", datamodule=datamodule, transformed=False, n_cols=2)

[INFO - AIXD.plotter] Overwriting datamodule with the provided one: {Train dataloader: size=800}
{Validation dataloader: size=100}
{Test dataloader: size=100}
{Predict dataloader: None}


/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 800 for training dataloader.

/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 100 for validation dataloader.

/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 100 for testing dataloader.

/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 100 for validation dataloader.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


Another helpful visualisation is gained by the evaluate_training() plotter function.

In [42]:
plotter.evaluate_training(datamodule=datamodule, attributes=["max_disp_cm"], transformed=False, bottom_top=(0.1, 0.9))

/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 100 for validation dataloader.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [44]:
plotter.evaluate_training(datamodule=datamodule, attributes=["Nfrequ1"], transformed=False, bottom_top=(0.1, 0.9))

/usr/local/lib/python3.12/dist-packages/aixd/mlmodel/data/data_loader.py:389: UserWarning:

Batch size was adjusted from 1000 to 100 for validation dataloader.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


### 5. Model deployment

Once the model is trained and evaluated, it can be deployed to the respective design problem.
Below we show how the trained model can be used to (i) inversely generate new samples based on defined constraints and (ii) evaluate a large amount of samples in quasi-real time.

##### (i) Inverse Design: Constrained Generation

Our goal here is to use the trained model to generate new samples, while considering defined performance constraints.

We here plot again the data distributions to know which perfromance ranges make sense to be requested.


In [45]:
plotter = Plotter(dataset=dataset, model=cae, datamodule=datamodule)

[INFO - AIXD.plotter] 
Plotter: Information
--------------------
The following block names and variable names are available as arguments for plotting methods:
    Block "design_parameters" (Design Parameters):
    -> Variables: "A", "mu", "sigma", "num_piers", "pier_h", "shell_cross_section", "pier_diameter", "Concrete"
    Block "performance_attributes" (Performance Attributes):
    -> Variables: "Nfrequ1", "Nfrequ2", "Nfrequ3", "max_disp_cm", "Mass_Kg", "util_average"
    Block "inputML" (Input ML):
    -> Variables: "A", "mu", "sigma", "num_piers", "pier_h", "shell_cross_section", "pier_diameter", "Concrete"
    Block "outputML" (Output ML):
    -> Variables: "Nfrequ1", "Nfrequ2", "Nfrequ3", "max_disp_cm", "Mass_Kg", "util_average"
Data from dataset blocks will be plotted in the original data domain.
Data from datamodule blocks will be plotted in the transformed domain (if `transformed=True`) or in the original domain, but after transformations (if `transformed=False`).



In [47]:
from aixd.mlmodel.generation.generator import Generator

gen = Generator(model=cae, datamodule=datamodule, over_sample=1)

[INFO - AIXD.mlmodel-generator] 
Generator: Information
----------------------
To request designs, the following variables are available:
    "Nfrequ1", "Nfrequ2", "Nfrequ3", "max_disp_cm", "Mass_Kg", "util_average"
The generation process is set to: Fast
    In this case, the values of z will not be generated conditioned on the y requested.
    This vastly accelerates the generation process, and is compensated by the over_sample
    parameter, which will allow generating more designs to then choose the best.
                



To make an example we want to generate samples which have an $Y_{2}$-value between 150 and 155. To execute this generation we first need to formulate this request in a dictionary, and then provide it to the generator. Here we chose to generate 1000 samples, and the output type DataFrame.

In [48]:
request = {"max_disp_cm": [1, 20]}

df_gen, _ = gen.generate(request, n_samples=1000, format_out="df")
df_gen

[INFO - AIXD.mlmodel-generator] 
Generator: Accuracy
-------------------
Requested attributes:               |              max_disp_cm |
Requested values:                   |               [1.0-20.0] |
Best generated sample:              |       17.864758994659642 |
Mean error of generated samples:    |                    ----- |
   .. of all returned 1000 samples: |   +/- 181.34425956757053 |
        .. of the best 100 samples: |   +/- 10.331183606727837 |




/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


,A,mu,sigma,num_piers,pier_h,shell_cross_section,pier_diameter,Concrete,Nfrequ1,Nfrequ2,Nfrequ3,max_disp_cm,Mass_Kg,util_average
26,3.728608,0.394091,0.710652,0,-1.488596,17.927222,50.231097,C60/75,1.458272,1.712859,2.898784,17.864759,188259.074909,0.628122
108,3.075976,0.402366,0.333353,0,-1.764162,18.548644,44.675367,C30/37,0.68669,0.974951,1.273581,6.833637,180125.054094,0.663107
356,2.721122,0.408253,0.306895,1,-3.018595,13.95826,28.472136,C60/75,0.98721,1.345092,1.894844,13.76517,126180.524103,0.425595
442,3.170207,0.339456,0.289862,0,-2.397433,12.083468,28.722926,C60/75,0.73862,0.914713,1.28649,5.802547,115041.304196,0.320973
865,3.634376,0.33426,0.268109,1,-2.423544,18.088066,28.32507,C30/37,1.17735,1.667022,2.660554,16.970949,184348.667712,0.775264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,1.962157,0.605518,0.462882,5,-0.231578,24.639764,52.053065,C40/50,3.976747,6.068608,8.017521,1048.676969,244257.804739,7.315395
917,1.557863,0.605797,0.586333,5,-0.150272,24.192799,53.835361,C40/50,4.182481,6.269859,8.295886,1097.589007,244071.017917,6.225708
581,0.86325,0.665452,0.713324,6,-0.017211,23.637687,57.01817,C40/50,4.567631,6.652597,8.810393,1203.909798,236981.26386,4.65863
469,0.542137,0.655379,0.80103,5,-0.669714,20.675316,43.905308,C40/50,3.256624,5.372527,7.107731,1244.758947,170928.345463,7.842548


Furthermore, we can not only condition on a single attribute, but on multiple attributes at the same time.

In [49]:
request = {"max_disp_cm": [0, 10], "Nfrequ1": [3, 5]}

df_gen, _ = gen.generate(request, n_samples=1000, format_out="df")
df_gen

[INFO - AIXD.mlmodel-generator] 
Generator: Accuracy
-------------------
Requested attributes:               |              max_disp_cm |                  Nfrequ1 |
Requested values:                   |               [0.0-10.0] |                [3.0-5.0] |
Best generated sample:              |       153.17267200954726 |       3.1071174114942552 |
Mean error of generated samples:    |                    ----- |                    ----- |
   .. of all returned 1000 samples: |   +/- 388.74546906110106 |   +/- 0.8942628459304561 |
        .. of the best 100 samples: |   +/- 260.53223562156205 |                  +/- 0.0 |




/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


,A,mu,sigma,num_piers,pier_h,shell_cross_section,pier_diameter,Concrete,Nfrequ1,Nfrequ2,Nfrequ3,max_disp_cm,Mass_Kg,util_average
840,2.880927,0.638562,0.645031,5,-4.266009,24.53148,59.107876,C60/75,3.107117,4.754766,7.191176,153.172672,267146.735092,2.083736
513,2.747718,0.654438,0.822043,4,-3.312194,23.356129,58.965673,C60/75,3.34696,4.929906,7.400484,157.374257,244725.88558,2.115393
449,2.187316,0.674522,0.712073,5,-4.485978,23.442785,59.123187,C60/75,3.15546,4.620213,7.109768,161.335965,244605.945312,1.959952
337,2.645049,0.505493,0.715697,3,-2.758701,19.722797,48.853188,C60/75,3.029601,4.593475,6.43693,162.366654,197297.529021,2.082246
997,4.824835,0.319952,0.876235,3,-0.191134,21.007701,44.54632,C60/75,3.368821,5.563321,7.12702,164.880932,252734.997671,3.117634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,4.969205,0.128461,0.706058,0,-0.0428,17.896406,36.822962,C60/75,1.367892,1.654514,2.509406,12.056187,208951.095487,0.605
420,4.737227,0.33544,0.422406,1,-1.048359,24.913294,26.620038,C30/37,1.363939,2.298607,3.991556,13.773337,286109.519962,1.074775
510,4.978597,0.129978,0.714001,0,-0.040688,16.803122,32.852345,C60/75,1.293529,1.520482,2.323827,10.300929,194062.628432,0.544397
909,4.93647,0.188392,0.530394,0,-0.063995,19.379277,35.489122,C30/37,1.187762,1.849946,2.930147,11.710563,229871.20015,0.911542


##### (ii) Forward Design: Predictions

We can also use the encoder model to evaluate a large amount of samples in quasi-real time. This can be beneficial for example for the optimisation of a design.


To show this functionality we here import a large amount of design parameter vectors from a csv file to be evaluated by the trained encoder


In [56]:
df_pred = cae.forward_evaluation(df_samples[:5].to_numpy(), format_out="df", input_transformed=False, return_untransformed=True)
df_pred

,Nfrequ1,Nfrequ2,Nfrequ3,max_disp_cm,Mass_Kg,util_average
0,1.000846,1.468982,1.979169,23.899678,163590.507592,1.053034
1,1.121386,1.917008,3.042596,23.138305,309943.619139,1.646884
2,2.454679,4.378053,6.662350,109.537415,206555.127870,1.917984
3,2.877634,4.590155,6.806800,546.343450,202079.314271,4.802455
4,0.929791,1.550625,2.332053,158.181186,118314.184901,1.437870


### 6.  Sensitivity analysis

The sensitivity of performance attributes at different design points is evaluated using the local and global sensitivity analysis features. In the first case, the sensitivity of user specified performance attributes is calculated at specific points in the design space, while in the context of global sensitivity analysis, the distribution of sensitivities is extracted from a number of different design points.

The extraction of sensitivities, requires the specification of the following inputs:

- the design point(s), at which the sensitivity is calculated
- the performance attribute, for which the sensitivity with respect to all design parameters is calculated

In [51]:
from aixd.mlmodel.sensitivity.sensitivities import LocalSensitivity, GlobalSensitivity
import torch

##### 6.1 Local Sensitivity

In [52]:
local_sensitivity = LocalSensitivity(cae)

# Calculating the local sensitivity
x = torch.tensor(datamodule.x_test, requires_grad=True)
x_truncated = x[5:6, :].detach().requires_grad_(True)
sensitivities = local_sensitivity.calculate(x_truncated, ["max_disp_cm", "Nfrequ1"])

# Plotting the local sensitivity
local_sensitivity.plot(x_truncated, "max_disp_cm")

# Plotting the local sensitivity
x_truncated = x[11:12, :].detach().requires_grad_(True)
local_sensitivity.plot(x_truncated, "Nfrequ1")

##### 6.2 Global Sensitivity

In [53]:
global_sensitivity = GlobalSensitivity(cae)

# Calculating the global sensitivity
x = torch.tensor(datamodule.x_test, requires_grad=True)
global_sensitivity_max_disp = global_sensitivity.calculate(x, "max_disp_cm")

# Plotting the global sensitivity of MaxDisp and Nfrequ1
global_sensitivity.plot(x, ["max_disp_cm", "Nfrequ1"])